In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [3]:
url = "https://www.amazon.eg/s?k=%D9%82%D8%B7%D8%B7&crid=1W9DGJBWD56SO&sprefix=%D9%82%D8%B7%2Caps%2C169&ref=nb_sb_noss_2"
response = requests.get(url, headers={
    'User-Agent': 'Mozilla 5 (Educational)'
})
print('contains listitem is ', response.text.count('listitem'))


contains listitem is  72


In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
listitems = soup.find_all('div', attrs={
    'role':'listitem'
})
print(len(listitems))
print(listitems[0])

72
<div class="sg-col-4-of-4 sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-8 sg-col-4-of-20" data-asin="B0CTSR7CM2" data-component-type="s-search-result" data-index="2" data-uuid="31cdb338-4f7f-4c2c-aebd-79f2b52ce470" id="31cdb338-4f7f-4c2c-aebd-79f2b52ce470" role="listitem"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-2" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-csa-c-item-id="amzn1.asin.1.B0CTSR7CM2" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render="">
<div class="rush-component s-expand-height" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-eu.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1769611538&amp;id=7339421894724235&amp;widgetName=sp_atf&amp;a

In [5]:
def clean_text(s):
    return re.sub(r'[\u200e\u200f\xa0]', ' ', s).strip()

In [11]:
items = []
for listitem in listitems:
    item = {}
    
    # Title
    title = listitem.find('div', attrs={
        'data-cy':'title-recipe'
    })
    if title:
        item['title'] = title.text
    
    # Price
    price = listitem.find('div', attrs={
        'data-cy':'price-recipe'
    })
    if(price):
        symbol = clean_text(price.select_one('span.a-price-symbol').text)
        whole = clean_text(price.select_one('span.a-price-whole').text)
        fraction = clean_text(price.select_one('span.a-price-fraction').text)
        # print('whole', whole)
        # print('fraction', fraction)
        priceText = f'{whole}{fraction}'
        priceText = priceText.replace(' ', '')
        priceText = priceText.replace(',', '')
        item['price'] = float(priceText)
        item['currency'] = symbol
    
    # Badge (ex: Best seller)
    badge = listitem.select_one('div#amazonBadges-capsule-view div div div')
    if badge:
        # inside inside the badge (ex: id="BEST_SELLER" inside it) 
        # .. assuming one badge for each .. if more 1 is taken
        badgeParts = list(badge.children)
        badgeTitle = badgeParts[0].text
        badgeAmong = badgeParts[1].text
        item['badge'] = badgeTitle
        item['badge among'] = badgeAmong

    # Image
    img = listitem.find('div', attrs={
        'data-cy': 'image-container'
    })
    if(img):
        img = img.select_one('img')
    if(img):
        item['image'] = img.attrs['src']
    else:
        item['image'] = None
    # print(item.get('image'))

    items.append(item)
    print(item)  

{'title': 'إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  باب أمان رفيع منزلق مقاوم للطقس للقطط والحيوانات الأليفة وللنوافذ والأبواب مع قفل رباعي الاتجاهات من تراسل، مقاس كبير ', 'price': 3745.0, 'currency': 'جنيه', 'image': 'https://m.media-amazon.com/images/I/51reYMX6vZL._AC_UL320_.jpg'}
{'title': 'إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  SARAR دراي فود للقطط المعقمة يوريناري كير بنكهة السلمون والتوت البري | 30% بروتين و12% دهون | دعم الكلى والجهاز البولي | 2 كجم ', 'price': 1250.0, 'currency': 'جنيه', 'image': 'https://m.media-amazon.com/images/I/51ZVs5WyOwL._AC_UL320_.jpg'}
{'title': 'إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة المنتج باستعلام البحث الذي قمت به.ترك تعليقات بشأن الإعلان  طعام رطب للقطط من بلوبو، 415 جم (عبوة من 24 قطعة، لحم بقري) ', 'price': 1560.0, 'currency': 'جنيه', 'image': 'https://m.media-amazon.com/images/I/51EMHndXdbL._AC_UL320_.jpg'}
{'

In [12]:
df = pd.DataFrame(items)

In [13]:
print(df)

                                                title   price currency  \
0   إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة ا...  3745.0     جنيه   
1   إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة ا...  1250.0     جنيه   
2   إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة ا...  1560.0     جنيه   
3   إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة ا...  1200.0     جنيه   
4   فرشاة شعر القطط من نيكوبي، فرشاة بخار للقطط، ف...   106.0     جنيه   
..                                                ...     ...      ...   
67                                                NaN     NaN      NaN   
68                                                NaN     NaN      NaN   
69                                                NaN     NaN      NaN   
70                                                NaN     NaN      NaN   
71                                                NaN     NaN      NaN   

                                                image          badge  \
0   https://m.media-amazon.com/images/I

In [14]:
df.describe(include='all')

,title,price,currency,image,badge,badge among
count,60,60.000000,60,60,9,9
unique,60,NaN,1,60,1,8
top,إعلانإعلان أنت ترى هذا الإعلان بناءً على صلة ا...,NaN,جنيه,https://m.media-amazon.com/images/I/51reYMX6vZ...,الأكثر مبيعاً,في Pet Supplies
freq,1,NaN,60,1,9,2
mean,NaN,490.304667,NaN,NaN,NaN,NaN
std,NaN,894.577061,NaN,NaN,NaN,NaN
min,NaN,39.600000,NaN,NaN,NaN,NaN
25%,NaN,87.017500,NaN,NaN,NaN,NaN
50%,NaN,174.500000,NaN,NaN,NaN,NaN
75%,NaN,263.250000,NaN,NaN,NaN,NaN


In [15]:
df.to_csv('amazon.csv')